In [3]:
import numpy as np
import sklearn
import pandas as pd
import pkgutil
import importlib
import tqdm

In [5]:
import evoapproxlib as eal
multipliers = [m.name for m in pkgutil.iter_modules(eal.__path__) if 'mul8s_' in m.name]
attrs = ['MAE_PERCENT', 'MAE', 'WCE_PERCENT', 'WCE', 'WCRE_PERCENT', 'EP_PERCENT', 'MRE_PERCENT', 'MSE', 'PDK45_PWR', 'PDK45_AREA', 'PDK45_DELAY']
for m in multipliers:
    print(m)

mul8s_1KV6
mul8s_1KV8
mul8s_1KV9
mul8s_1KVA
mul8s_1KVM
mul8s_1KVP
mul8s_1KVQ
mul8s_1KX5
mul8s_1KXF
mul8s_1L12
mul8s_1L2J
mul8s_1L2L
mul8s_1L2N


In [6]:
def multiplier_output(multiplier, x, y, signed=True, bitwidth=8):
    axmul = np.vectorize(multiplier.calc)
    z = axmul(x, y)
    if signed:
        z[z >= 2**(2*bitwidth-1)] -= 2**(2*bitwidth)
    return z

In [39]:
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

def sampled_features(bitwidth=16, num_samples=int(1e7)):
    rng = np.random.default_rng(42)
    x = np.sort(rng.integers(low=-2**(bitwidth-1), high=2**(bitwidth-1), size=num_samples))
    y = np.sort(rng.integers(low=-2**(bitwidth-1), high=2**(bitwidth-1), size=num_samples))
    return np.vstack([x, y]).T

def base_features(bitwidth=8):
    x = np.arange(-2**(bitwidth-1), 2**(bitwidth-1))
    xx, yy = np.meshgrid(x, x, indexing='ij')
    X = np.vstack([xx.flatten(), yy.flatten()]).T
    return X

def build_model(emap, expansions=[], prune=False, X=None, interaction_only=True):
    def mod_features(X):
        return np.hstack([X] + [X % sc for sc in expansions])

    # Fit over entire output space
    if X is None:
        X = base_features()

    # Build feature transformations
    # Add Modulo features
    features = mod_features(X)
    # Polynomial Expansion
    poly = PolynomialFeatures(degree=2, interaction_only=interaction_only)
    features = poly.fit_transform(features)

    # Build Model
    reg = linear_model.LinearRegression()
    reg = reg.fit(features, emap)
    
    # Drop small coefficients
    if prune:
        reg.coef_ = reg.coef_.round(decimals=1)

    class Predictor():
        def __init__(self, mod, poly, regressor):
            self.mod_transform = mod
            self.poly_transform = poly
            self.regressor = regressor

        def __call__(self, X):
            features = self.mod_transform(X)
            features = self.poly_transform.fit_transform(features)
            return self.regressor.predict(features)

    return Predictor(mod_features, poly, reg)

In [40]:
def wce(emap, model_output):
    ans = np.abs(emap-model_output)
    return np.max(ans)

def mae(emap, model_output):
    ans = np.abs(emap-model_output)
    return np.mean(ans)

def mre(emap, model_output):
    ans = np.abs(emap-model_output)
    div = np.maximum(np.ones_like(ans), ans)
    return np.mean(ans/div)

In [41]:
def metrics(emap, model, X):
    model_output = np.round(model(X).reshape(emap.shape))
    return {
        "wce" : wce(emap, model_output),
        "mre" : mre(emap, model_output),
        "mae" : mae(emap, model_output),
    }

results = []
bf = base_features()
expansions = [float(2**exp) for exp in range(1,10)]

for mname in tqdm.tqdm(multipliers):
    multiplier = importlib.import_module(f"evoapproxlib.{mname}")
    emap = multiplier_output(multiplier, bf[:,0], bf[:,1])
    evoapprox_metrics = dict([(a.lower(), getattr(multiplier, a)) for a in attrs])

    base = build_model(emap, X=bf, interaction_only=False)
    base_metrics = metrics(emap, base, bf)
    base_params = {
        'coefficients': list(base.regressor.coef_.tolist()),
        'powers': list(base.poly_transform.powers_.tolist()),
    }

    expansion = 0
    best_mae = base_metrics['mae']
    # Grid Search for j=k
    for e in expansions:
        ours = build_model(emap, expansions=[e], prune=True, X=bf)
        cur_mae = metrics(emap, ours, bf)['mae']
        print(cur_mae, best_mae)
        if cur_mae < best_mae:
            expansion = e
            best_mae = cur_mae

    ours = build_model(emap, expansions=[expansion], prune=True, X=bf)
    htp_metrics = metrics(emap, ours, bf)
    mask = ~np.isclose(ours.regressor.coef_, 0)
    htp_params = {
        'coefficients': list(ours.regressor.coef_[mask].tolist()),
        'powers': list(ours.poly_transform.powers_[mask].tolist()),
        'expansion' : expansion,
    }

    results.append({
        'name' : mname,
        'evoapprox_metrics' : evoapprox_metrics,
        'base_params' : base_params,
        'htp_params' : htp_params,
        'base_metrics' : base_metrics,
        'htp_metrics' : htp_metrics,
    })

  0%|          | 0/13 [00:00<?, ?it/s]

0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0


/tmp/ipykernel_5621/874660020.py:19: RuntimeWarning: divide by zero encountered in remainder
  return np.hstack([X] + [X % sc for sc in expansions])
/tmp/ipykernel_5621/874660020.py:19: RuntimeWarning: divide by zero encountered in remainder
  return np.hstack([X] + [X % sc for sc in expansions])
  8%|▊         | 1/13 [00:01<00:12,  1.01s/it]

0.75 1.25
0.625 0.75
1.140625 0.625
1.25 0.625
1.25 0.625
1.25 0.625
1.25 0.625
1.25 0.625
1.25 0.625


 15%|█▌        | 2/13 [00:01<00:10,  1.03it/s]

...

 85%|████████▍ | 11/13 [00:10<00:01,  1.10it/s]

85.3592529296875 92.46533203125
0.0 85.3592529296875
79.75143432617188 0.0
93.28233337402344 0.0
93.64703369140625 0.0
92.79067993164062 0.0
92.79067993164062 0.0
92.79067993164062 0.0
92.79067993164062 0.0


 92%|█████████▏| 12/13 [00:10<00:00,  1.09it/s]

185.085205078125 188.51007080078125
170.873046875 185.085205078125
0.0 170.873046875
164.7843780517578 0.0
186.84445190429688 0.0
191.91400146484375 0.0
189.19464111328125 0.0
189.8656768798828 0.0
189.8656768798828 0.0


100%|██████████| 13/13 [00:11<00:00,  1.09it/s]


In [37]:
for r in results:
    print(r['name'], r['base_metrics'], r['htp_metrics'])

mul8s_1KV6 {'wce': 0.0, 'mre': 0.0, 'mae': 0.0} {'wce': 0.0, 'mre': 0.0, 'mae': 0.0}
mul8s_1KV8 {'wce': 4.0, 'mre': 0.9375, 'mae': 1.25} {'wce': 2.0, 'mre': 0.5625, 'mae': 0.625}
mul8s_1KV9 {'wce': 13.0, 'mre': 0.8125, 'mae': 3.15625} {'wce': 6.0, 'mre': 1.0, 'mae': 2.15625}
mul8s_1KVA {'wce': 38.0, 'mre': 0.92822265625, 'mae': 7.8887939453125} {'wce': 19.0, 'mre': 0.953125, 'mae': 5.34375}
mul8s_1KVM {'wce': 65.0, 'mre': 0.9921875, 'mae': 31.999176025390625} {'wce': 0.0, 'mre': 0.0, 'mae': 0.0}
mul8s_1KVP {'wce': 72.0, 'mre': 0.992218017578125, 'mae': 32.067413330078125} {'wce': 6.0, 'mre': 0.875, 'mae': 2.0}
mul8s_1KVQ {'wce': 90.0, 'mre': 0.99224853515625, 'mae': 32.50360107421875} {'wce': 22.0, 'mre': 0.9375, 'mae': 5.875}
mul8s_1KX5 {'wce': 245.0, 'mre': 0.9954071044921875, 'mae': 65.33404541015625} {'wce': 40.0, 'mre': 0.96875, 'mae': 11.0}
mul8s_1KXF {'wce': 463.0, 'mre': 0.9951171875, 'mae': 127.879638671875} {'wce': 0.0, 'mre': 0.0, 'mae': 0.0}
mul8s_1L12 {'wce': 4548.0, 'mre'

In [38]:
import json
with open('mul8s_models.json', 'w+') as f:
    json.dump(results, f, indent=4)